In [6]:
import pandas as pd
import numpy as np
xlsx = pd.read_csv('vrp_ekz_prog.csv')
l = (xlsx['o_potato_atyrau'].shift(1) + xlsx['o_grain_atyrau'].shift(1) + xlsx['o_vegetables_atyrau'].shift(1))
l[0] = 44.15

Gplg = ((xlsx['o_potato_atyrau'] + xlsx['o_grain_atyrau'] + xlsx['o_vegetables_atyrau'])/l)

newdf = pd.DataFrame(columns=['c1'], index=range(21))

c=100
i=0
while i<=20:
    c = (c*Gplg[i])
    newdf.loc[i].c1 = c
    i=i+1
GplgS=newdf['c1']

GplgS

0     107.995
1     116.648
2     126.025
3     136.127
4     147.089
5     158.935
6     171.755
7     107.995
8     116.648
9     126.025
10    136.127
11    147.089
12    158.935
13    171.755
14    116.648
15    126.025
16    136.127
17    147.089
18    158.935
19    171.755
20    107.995
Name: c1, dtype: object